In [1]:
from IPython.display import display_html
display_html("""
<div style="text-align:center; margin: 20px 0;">
<p style="color:#FA5882;text-align:center; margin: 10px 0 20px 0;">

</p>
<button onclick="$('.input, .prompt, .output_stderr, .output_error').toggle();">PE-TTM PB-TTM因子双因子分析</button>
<hr/>
</div>
""", raw=True)

PE-TTM PB-TTM因子双因子分析

In [2]:
import sys
import os
import datetime
from collections import OrderedDict
import pandas as pd
import numpy as np
import pyfolio as pf
from pyfolio import plotting
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
import seaborn as sns
from matplotlib.ticker import FuncFormatter

project_path = os.path.abspath(os.path.dirname(os.getcwd())+os.path.sep+".")
sys.path.append(project_path)
from FactorAnalyse import FactorAnalyserBase
from DataPreProcessing import *

figsize=(20, 9)

D:\Anaconda3\envs\QTA\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
import logging
from sklearn.metrics import mean_squared_error
from Tool.logger import Logger
from Tool.DataPreProcessing import DeExtremeMethod, ImputeMethod, StandardizeMethod
from BackTesting.Signal.SignalSynthesis import SignalSynthesis
from BackTesting.systhesisDirector import SignalDirector

np.warnings.filterwarnings('ignore')

logger = Logger("SignalDirector")
logger.setLevel(logging.WARNING)

params = {
    "startDate": pd.to_datetime('20170301'),
    "endDate": pd.to_datetime('20201231'),
    "panelSize": 5,
    "trainTestGap": 1,
    "maskList": None,
    "deExtremeMethod": DeExtremeMethod.MeanStd(),
    "imputeMethod": ImputeMethod.JustMask(),
    "standardizeMethod": StandardizeMethod.StandardScaler(),
    "pipeline": None,
    "factorNameList": ['close', 'amount', 'open', 'high', 'low'],
    # params for XGBoost
    "modelParams": {
        "jsonPath": None,
        "paraDict": {
            "n_estimators": 50,
            "random_state": 42,
            "max_depth": 2}
    },
    # metric function for machine learning models
    "metric_func": mean_squared_error,
    # smoothing params
    "smoothing_params": None
}

director = SignalDirector(SignalSynthesis, params=params, logger=logger)

# 定义策略

这是一个简单的demo策略。
#### 去掉以下条件的股票：
1. PE-TTM<=0


In [4]:
class SampleStrategy(FactorAnalyserBase):
    def __init__(self, start_date, end_date, benchmark_id, universe, props):
        super(SampleStrategy, self).__init__(director, start_date, end_date, benchmark_id,
                                             universe, props)

    def filter(self):
        for date, df in self.raw_universe_df_dict.items():
            df['上市天数'] = (date-df['ipo_date']).dt.days + 1  # 自然日
            self.set_filter(df['is_trading'].astype(bool) & (df['上市天数'] > 180) & df['is_exist'], date)

    def rate_stock(self):
        """
        选股逻辑，去极值、中性化、标准化等。需要用户自己定义
        """
        for date, df in self.processed_universe_df_dict.items():
            score = self.signals.loc[date, df.index.to_list()]
            self.set_score(score, date)

# 回测

#### PETTM_PBTTM_Strategy类benchmark参数
1. 可选'000300.SH', '000905.SH' 

#### PETTM_PBTTM_Strategy类universe参数
1. 可选'全A'、'沪深300'、'中证500'

#### PETTM_PBTTM_Strategy类props参数：
1. 不另行设置时默认值为：props = {'单边交易费率': 0.0015, '换仓日期模式': '月初换', '自定义换仓日期': None, '自定义篮子': None}  
2. 换仓日期模式可选 月初换，月末换和自定义
3. 当换仓日期模式选择了自定义，需要同时向自定义换仓日期传入换仓日期list
4. 自定义篮子尚不支持，占坑。  

#### grouping_test方法介绍

分组收益分析，基于Score分组，由大到小分别是Q1、Q2...Q group_num，快速回测  
        :param group_num: 分组数量  
        :param control_dict: 需要控制的因子，若不为空必须是OrderDict，需要控制的因子名称为key，分组数量为value，
                              非数字型因子，如行业分类，则value为空字符串 ： ""。将按顺序依次控制分组。
                              例如：OrderedDict([('申万一级行业', ''), ('流通市值', 3)])  
        :param group_by_benchmark: 是否按基准的因子去划分  
        :param weight_method:配权方法，目前只支持EW、LVW和VW，分别是等权、流通市值平方根加权，总市值平方根加权  
        :param max_stock_num 每个分组网格内最大的持股数  
        :return: result_dict:包含4个部分。分组回测的净值、分组回测每个持有周期的IC、分组回测每次的换手率、
                                            控制变量下每组的股票数量。  

In [5]:
fab = SampleStrategy(params['startDate'], params['endDate'], '000905.SH', '全A',
                           {"换仓日期模式":"每日换", '单边交易费率': 0.})
fab.prepare_data()
fab.filter()
fab.rate_stock()

# start_ = datetime.datetime.now()

# print(datetime.datetime.now()-start_)
# result = fab.grouping_test(5, OrderedDict([('industry_zx1_name', ''), ('circulating_market_cap', 3)]),
#                                group_by_benchmark=True, weight_method='LVW')


2021-01-17 21:32:43,683 logger.py[line:40] - INFO - root : globalVars is initialized
2021-01-17 21:32:43,684 logger.py[line:40] - INFO - root : materialData:{} is now in global
2021-01-17 21:32:45,151 logger.py[line:40] - INFO - root : close is now in globalVars.materialData
2021-01-17 21:32:46,192 logger.py[line:40] - INFO - root : high is now in globalVars.materialData
2021-01-17 21:32:47,233 logger.py[line:40] - INFO - root : low is now in globalVars.materialData
2021-01-17 21:32:50,231 logger.py[line:40] - INFO - root : open is now in globalVars.materialData
2021-01-17 21:32:53,634 logger.py[line:40] - INFO - root : preclose is now in globalVars.materialData
2021-01-17 21:32:56,807 logger.py[line:40] - INFO - root : amount is now in globalVars.materialData
2021-01-17 21:33:00,446 logger.py[line:40] - INFO - root : volume is now in globalVars.materialData
2021-01-17 21:33:04,097 logger.py[line:40] - INFO - root : pctChange is now in globalVars.materialData
2021-01-17 21:33:07,562 lo

FactorProfileBase __init__
close is now in globalVars.factors
FactorProfileBase __init__
amount is now in globalVars.factors
FactorProfileBase __init__
open is now in globalVars.factors
FactorProfileBase __init__
high is now in globalVars.factors
FactorProfileBase __init__
low is now in globalVars.factors


  0%|          | 0/939 [00:00<?, ?it/s]

2021-01-17 21:33:33,029 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) training loss: 2.520237409098544, testing loss: 3.8999928889452224
2021-01-17 21:33:33,412 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_r

2021-01-17 21:33:37,609 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) training loss: 2.6155441171035085, testing loss: 5.727365036900835
2021-01-17 21:33:37,990 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_r

...

2021-01-17 21:38:08,594 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) training loss: 5.251289436483547, testing loss: 10.20686792939819
2021-01-17 21:38:08,867 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_ra

2021-01-17 21:38:12,144 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) training loss: 5.941548606550966, testing loss: 7.192356107754215
2021-01-17 21:38:12,435 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_ra

2021-01-17 21:38:15,530 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) training loss: 4.971268980893109, testing loss: 5.73345900227905
2021-01-17 21:38:15,818 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rat

2021-01-17 21:38:18,953 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) training loss: 6.302713174544177, testing loss: 7.07054285132482
2021-01-17 21:38:19,248 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rat

2021-01-17 21:38:22,246 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=8, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None) training loss: 4.806897762285364, testing loss: 6.71387163608035
2021-01-17 21:38:22,523 logger.py[line:40] - INFO - root : Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rat

In [6]:
result = fab.grouping_test(5, OrderedDict([('circulating_market_cap', 5)]),
                               group_by_benchmark=True, weight_method='VW')
# result = fab.grouping_test(5, {}, weight_method='VW')

0it [00:00, ?it/s]

ValueError: Length of values (39) does not match length of index (38)

# 分析结果

## 分组净值


In [ ]:
nav_df = (1+result.all_group_ret_df).cumprod()
nav_df[result.group_list+[result.benchmark_id]].plot(figsize=figsize,fontsize=15)

## 分组累计超额

整个回测区间的累积超额收益率

In [ ]:
excess_df = nav_df.iloc[-1,:-1] - nav_df.iloc[-1,-1]
excess_df[result.group_list].plot(kind='bar', figsize=figsize,fontsize=25, color='dodgerblue',grid=True)

## 多空组合净值

In [ ]:
nav = nav_df[result.group_list[0]] - nav_df[result.group_list[-1]] + 1

plt.figure(figsize=figsize)
plotting.plot_drawdown_periods(
        nav.pct_change(), top=1, fontsize=15)
plt.title('多空组合净值',fontsize=15)

## 指数减空头

In [ ]:
nav = nav_df[result.benchmark_id] - nav_df[result.group_list[-1]] + 1

plt.figure(figsize=figsize)
plotting.plot_drawdown_periods(
        nav.pct_change(), top=1, fontsize=15)
plt.title('指数减空头',fontsize=15)

## 多头减指数

In [ ]:
nav = nav_df[result.group_list[0]] - nav_df[result.benchmark_id] + 1

plt.figure(figsize=figsize)
plotting.plot_drawdown_periods(
        nav.pct_change(), top=1, fontsize=15)
plt.title('多头减指数',fontsize=15)

## 年度统计

以下分析都是针对Q1组合的。  
注1：年化因子为365天，与wind一致。  
注2：超额收益即 Rp - Rb  
注3：alpha 和 beta 是根据CAMP公式将Rp-Rf与Rb-Rf进行回归得出的系数。因此alpha相比超额收益来说多了风险调整这一步。  

In [ ]:
annual_ra_df = result.get_annual_return_statistic()
styles = [
            dict(selector="caption", props=[("font-size", "150%"),
                                            ("text-align", "center")]),
            dict(selector="caption", props=[("caption-side", "bottom")]),
            dict(selector="th", props=[("max-width", "1200px"),
                                       ("text-align", "center")])]
pct_fields = []
float_fields = []
for col in annual_ra_df:
    if '%' in col:
        pct_fields.append(col)
    else:
        float_fields.append(col)
annual_ra_df.style.format("{:.2%}", subset=pct_fields) \
            .format("{:.2}", subset=float_fields) \
            .set_properties(**{'max-width': '1000px', 'font-size': '11pt'}) \
            .set_caption("净值评价") \
            .set_table_styles(styles) \
            .bar(align='mid', color=['#5fba7d', '#d65f5f'])

## 纯多月收益统计

以下分析都是针对Q1组合的。

In [ ]:
plt.figure(figsize=(15,6))
ax_monthly_heatmap = plt.subplot(121)
ax_monthly_dist = plt.subplot(122)
plotting.plot_monthly_returns_heatmap(result.ret_df['p'], ax=ax_monthly_heatmap)
plotting.plot_monthly_returns_dist(result.ret_df['p'], ax=ax_monthly_dist)

## 超额月收益统计

以下分析都是针对Q1组合的。

In [ ]:
plt.figure(figsize=(15,6))
ax_monthly_heatmap = plt.subplot(121)
ax_monthly_dist = plt.subplot(122)
plotting.plot_monthly_returns_heatmap(result.ret_df['excess'], ax=ax_monthly_heatmap)
plotting.plot_monthly_returns_dist(result.ret_df['excess'], ax=ax_monthly_dist)

## 持有期IC

IC分析不仅针对Q1，下图中每个柱子是直接对universe里面的所有股票的score与该持有期内的累计收益率作秩相关系数。  
如果回测中设定了“换仓日期模式”为 “月初换”或“月末换”，那么持有期就是一个自然月。而如果“换仓日期模式”是“自定义”，那持有期就是自定义日期序列里两个相邻日期之间的时间段。

In [ ]:
from scipy import stats
ic_plus = result.ic_series.copy()
ic_plus[result.ic_series<0] = np.nan
ic_minus = result.ic_series.copy()
ic_minus[result.ic_series>=0] = np.nan

fig = plt.figure(figsize=(20,20))
ax1 = fig.add_subplot(311)
ic_plus.plot(kind='bar', color='r',ax=ax1, fontsize=15)
ic_minus.plot(kind='bar', color='g',ax=ax1, fontsize=15)

ax2 = fig.add_subplot(312)
result.ic_series.plot(kind='kde', ax=ax2)

fig.add_subplot(313)
a=stats.probplot(result.ic_series, dist="norm", plot=plt)

## 回归分析

每个持有期内，对全universe的期初score与当期收益率作回归，但这里还考虑了行业影响，加入了行业哑变量，如下：
$$ R = \alpha + \beta_1 score + \sum_{i=1}^{n}{\beta_{2,i}\times SWInd_i} + \epsilon$$
其中SWInd_i是行业哑变量。

In [ ]:
result.regression_result_df.astype(float).hist(bins=30, figsize=figsize)
result.regression_result_df.astype(float).describe()

## 行业超额收益率（中位数）

各行业在每个持有期内超额收益的中位数

In [ ]:
result.industry_alpha_df.median(axis=1).sort_values(ascending=False).plot(kind='bar',color='dodgerblue', figsize=figsize)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

## 持仓换手率

In [ ]:
result.turnover_rate_df[result.group_list].plot(figsize=figsize, fontsize=15)
plt.ylabel('换手率', fontsize=15)
result.turnover_rate_df[result.group_list].astype(float).describe()

## 各分组内的可选股票数量

In [ ]:
result.result_count_series_dict[max(result.result_count_series_dict.keys())]

## score分布

In [ ]:
fig=plt.figure(figsize=figsize)
# 第一期
fig.add_subplot(121)
fab.processed_universe_df_dict[min(fab.processed_universe_df_dict.keys())]['score'].hist(bins=100)
# 最后一期
fig.add_subplot(122)
fab.processed_universe_df_dict[max(fab.processed_universe_df_dict.keys())]['score'].hist(bins=100)

l = []
for date,df in fab.processed_universe_df_dict.items():
    l.append(df['score'])
concat_df = pd.concat(l, axis=1)

fig1=plt.figure(figsize=figsize)
ax = fig1.add_subplot(111)
concat_df.plot(kind='box', ax=ax)

## 数字型因子分布

可以把factor_name换成流通市值以外的任何数字型因子，从而看到持仓组合和基准组合的因子分布差别。

In [ ]:
factor_name = 'free_circulating_market_cap'

fig=plt.figure(figsize=figsize)
# 第一期
ax1 = fig.add_subplot(221)
df = fab.raw_universe_df_dict[min(fab.raw_universe_df_dict.keys())]
df = df[df[result.group_list[0]] > 0]
first_period_df = pd.concat([df[factor_name],
                              fab.benchmark_weight_df_dict[min(fab.benchmark_weight_df_dict.keys())][factor_name]],
                            axis=1)
first_period_df.columns=('持仓'+factor_name, '基准'+factor_name)
first_period_df['持仓'+factor_name].plot(kind='kde', ax=ax1, fontsize=15) 
first_period_df['基准'+factor_name].plot(kind='kde', ax=ax1, fontsize=15) 
plt.legend()
plt.title('第一期{0}分布(普通坐标轴)'.format(factor_name))

ax2 = fig.add_subplot(222)
df = fab.raw_universe_df_dict[min(fab.raw_universe_df_dict.keys())]
df = df[df[result.group_list[0]] > 0]
first_period_df = pd.concat([df[factor_name],
                              fab.benchmark_weight_df_dict[min(fab.benchmark_weight_df_dict.keys())][factor_name]],
                            axis=1)
first_period_df.columns=('持仓'+factor_name, '基准'+factor_name)
first_period_df['持仓'+factor_name].plot(kind='kde', ax=ax2, logx=True, fontsize=15) 
first_period_df['基准'+factor_name].plot(kind='kde', ax=ax2, logx=True, fontsize=15) 
plt.legend()
plt.title('第一期{0}分布(对数坐标轴)'.format(factor_name))
# 最后一期
ax3 = fig.add_subplot(223)
df = fab.raw_universe_df_dict[max(fab.raw_universe_df_dict.keys())]
df = df[df[result.group_list[0]] > 0]
last_period_df = pd.concat([df[factor_name],
                              fab.benchmark_weight_df_dict[max(fab.benchmark_weight_df_dict.keys())][factor_name]],
                            axis=1)
last_period_df.columns=('持仓'+factor_name, '基准'+factor_name)
last_period_df['持仓'+factor_name].plot(kind='kde', ax=ax3, fontsize=15) 
last_period_df['基准'+factor_name].plot(kind='kde', ax=ax3, fontsize=15) 
plt.legend()
plt.title('最后一期{0}分布(普通坐标轴)'.format(factor_name))

ax4 = fig.add_subplot(224)
df = fab.raw_universe_df_dict[max(fab.raw_universe_df_dict.keys())]
df = df[df[result.group_list[0]] > 0]
last_period_df = pd.concat([df[factor_name],
                              fab.benchmark_weight_df_dict[max(fab.benchmark_weight_df_dict.keys())][factor_name]],
                            axis=1)
last_period_df.columns=('持仓'+factor_name, '基准'+factor_name)
last_period_df['持仓'+factor_name].plot(kind='kde', ax=ax4, logx=True, fontsize=15) 
last_period_df['基准'+factor_name].plot(kind='kde', ax=ax4, logx=True, fontsize=15) 
plt.legend()
plt.title('最后一期{0}分布(对数坐标轴)'.format(factor_name))

## 行业权重分布

In [ ]:
industry_factor_name = 'industry_zx1_name'

fig=plt.figure(figsize=(20,25))
# 第一期
ax1 = fig.add_subplot(211)
df = fab.raw_universe_df_dict[min(fab.raw_universe_df_dict.keys())]
df = df[df[result.group_list[0]] > 0]
bench_df = fab.benchmark_weight_df_dict[min(fab.benchmark_weight_df_dict.keys())]
first_period_df = pd.concat([df.groupby(industry_factor_name)[result.group_list[0]].sum(),
                             bench_df.groupby(industry_factor_name)['权重'].sum()],
                            axis=1)
first_period_df.columns = ('持仓'+industry_factor_name+'权重', '基准'+industry_factor_name+'权重')
first_period_df.sort_values('持仓'+industry_factor_name+'权重',inplace=True, ascending=False)
first_period_df.plot(kind='bar', ax=ax1, fontsize=15)
plt.title('第一期')
# 最后一期
ax1 = fig.add_subplot(212)
df = fab.raw_universe_df_dict[max(fab.raw_universe_df_dict.keys())]
df = df[df[result.group_list[0]] > 0]
bench_df = fab.benchmark_weight_df_dict[max(fab.benchmark_weight_df_dict.keys())]
last_period_df = pd.concat([df.groupby(industry_factor_name)[result.group_list[0]].sum(),
                             bench_df.groupby(industry_factor_name)['权重'].sum()],
                            axis=1)
last_period_df.columns = ('持仓'+industry_factor_name+'权重', '基准'+industry_factor_name+'权重')
last_period_df.sort_values('持仓'+industry_factor_name+'权重',inplace=True, ascending=False)
last_period_df.plot(kind='bar', ax=ax1, fontsize=15)
plt.title('最后一期')